In [ ]:
from database import FinanceDB

db = FinanceDB()
financas = db.get_financas()
accounts = db.get_accounts()

In [32]:
def money(value:float) -> str:
    return f"R$ {value:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

In [ ]:
def get_totals(df, inicial_balance=0, log=True):
    total_entrada = df[df["tipo"] == "Recebimento"]["valor"].sum()
    total_saida = -df[df["tipo"] == "Gasto"]["valor"].sum()
    total_transferencias = -df[df["tipo"] == "Transferência"]["valor"].sum()

    saldo = total_entrada - total_saida - total_transferencias + inicial_balance

    if log:
        print(f"Total Entrada: {money(total_entrada)}")
        print(f"Total Saída: {money(total_saida)}")
        print(f"Total Transferências: {money(total_transferencias)}")
        print(f"Saldo: {money(saldo)}")
        
    return total_entrada, total_saida, total_transferencias, saldo

In [42]:
totais = get_totals(financas, inicial_balance=accounts["saldo_inicial"].sum())

Total Entrada: R$ 45.259,78
Total Saída: R$ 24.077,82
Total Transferências: R$ 22.517,36
Saldo: R$ 1.664,60


In [57]:
def get_summary(df, period:str='M'):
    period = period.upper()
    types = {'Y':'ano', 'M': 'mês', 'W': 'semana', 'D': 'dia'}
    if period not in types:
        raise ValueError(f"Período inválido: {period}. Use  'Y' para anual, 'M' para mensal, 'W' para semanal ou 'D' para diário.")
    
    df[types[period]] = df["data"].dt.to_period(period)
    summary = df.groupby([types[period], "tipo"])["valor"].sum().unstack().fillna(0)
    summary.drop(columns=["Transferência"], inplace=True, errors='ignore')
    summary = summary[["Recebimento", "Gasto"]]
    summary["saldo"] = summary.get("Recebimento", 0) + df.get("Gasto", 0)
    return summary

In [59]:
resumo_mensal = get_summary(financas)
resumo_mensal

tipo,Recebimento,Gasto,saldo
mês,,,
2024-01,0.00,0.00,0.00
2024-02,0.00,0.00,0.00
2024-03,0.00,0.00,0.00
2024-04,0.00,0.00,0.00
2024-05,0.00,0.00,0.00
2024-06,0.00,0.00,0.00
2024-07,0.01,0.00,0.01
2024-08,1200.00,-3688.26,1200.00
2024-09,0.00,-511.39,0.00
